# Calculate prescales per year for each PFJet trigger

### Commands to run for brilcalc

```
brilcalc lumi --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json -u /nb -i Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt   --hltpath "HLT_PFJet*" --output-style csv > runs2016.csv
brilcalc lumi --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json -u /nb -i Cert_294927-306462_13TeV_UL2017_Collisions17_GoldenJSON.txt --hltpath "HLT_PFJet*" --output-style csv > runs2017.csv
brilcalc lumi --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json -u /nb -i Cert_314472-325175_13TeV_Legacy2018_Collisions18_JSON.txt   --hltpath "HLT_PFJet*" --output-style csv > runs2018.csv
```

These files were edited by hand to extract only the summary, remove the preceding pound sign (comment), and remove the `Fwd` jet triggers. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_prescales(year):
    name = ("summary%s.csv") % (year)
    df = pd.read_csv( name )
    alltrigs = []
    for i in df["hltpath"]: 
        pathname = '_'.join(i.split("_")[0:2])
        if pathname not in alltrigs:
            alltrigs.append(pathname)
    initvals = np.zeros_like(alltrigs, dtype="int")
    lumi_pertrig = dict(zip( alltrigs, initvals ))
    paths = df['hltpath']
    lumis = df['totrecorded(/nb)']
    pathnames = [ '_'.join(i.split("_")[0:2]) for i in paths ]
    for i in range(len(paths)):
        lumi_pertrig[pathnames[i]] += lumis[i]
    totlumi = max(lumi_pertrig.values()) 
    prescales_pertrig = {}
    for s,t in lumi_pertrig.items():
        prescales_pertrig[s] = np.where( t > 0, totlumi / t, 0)        
    return totlumi, prescales_pertrig

In [3]:
for year in ['2016', '2017', '2018']: 
    totlumi, prescales_pertrig = get_prescales(year)
    print ("Year : {0:4s}, Total lumi: {1:2.2f} fb-1".format(year, totlumi / 1000000))
    print("Prescales:")    
    trignames = prescales_pertrig.keys()
    trigthresholds = sorted([int(x[9:]) for x in trignames])
    for s in trigthresholds:
        sname = "HLT_PFJet" + str(s)
        print(" {0:20s} : {1:<12.2f}".format(sname, prescales_pertrig[ sname ] ) )
    print("\n\n")

Year : 2016, Total lumi: 36.31 fb-1
Prescales:
 HLT_PFJet40          : 136006.59   
 HLT_PFJet60          : 50007.75    
 HLT_PFJet80          : 13163.18    
 HLT_PFJet140         : 1501.12     
 HLT_PFJet200         : 349.82      
 HLT_PFJet260         : 61.17       
 HLT_PFJet320         : 20.49       
 HLT_PFJet400         : 6.99        
 HLT_PFJet450         : 1.00        
 HLT_PFJet500         : 1.00        



Year : 2017, Total lumi: 41.48 fb-1
Prescales:
 HLT_PFJet40          : 86061.17    
 HLT_PFJet60          : 36420.75    
 HLT_PFJet80          : 9621.74     
 HLT_PFJet140         : 1040.40     
 HLT_PFJet200         : 189.54      
 HLT_PFJet260         : 74.73       
 HLT_PFJet320         : 29.49       
 HLT_PFJet400         : 9.85        
 HLT_PFJet450         : 3.97        
 HLT_PFJet500         : 1.00        
 HLT_PFJet550         : 1.00        



Year : 2018, Total lumi: 59.83 fb-1
Prescales:
 HLT_PFJet15          : 318346231.66
 HLT_PFJet25          : 318346231.66
 H